In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_excel('restaurant_info_6Districts_Ju.xlsx')

<h1> Data clean

In [3]:
#remove titles
offender_index = df[df['happy']=="happy"].index
df= df.drop(offender_index)

#remove asia miles
asia_miles_index = df[df['food_type'].str.contains("eligible transactions")].index
df= df.drop(asia_miles_index)
df = df.reset_index()
df = df.drop('index',axis=1)

In [4]:
#drop duplicate entries with same name and location
df.drop_duplicates(subset=['name', 'location'], keep='last',inplace = True)

In [5]:
#change columns to float type
df['happy'] = df['happy'].astype(float)
df['sad'] = df['sad'].astype(float)
df['bookmark'] = df['bookmark'].astype(float)

In [6]:
#enable sort by price
df['price'] = pd.Categorical(df['price'], ['Below $50','$51-100', '$101-200', '$201-400', '$401-800', 'Above $801'])

In [7]:
#insert additional column of pure numeric prices
df['price_int'] = df['price'].replace({"Below $50":25,"$51-100":75,"$101-200":150,"$201-400":300,"$401-800":600,"Above $801":1000})


In [8]:
#define locations
Places = ['Causeway Bay','Central',"Yuen Long","Tsuen Wan","Tsim Sha Tsui","Mong Kok"]

In [9]:
#give food type a number
food_model = OrdinalEncoder()
test = pd.DataFrame(food_model.fit_transform(df[['food_type']]))

df = pd.concat([df,test],axis=1,join='inner')
df = df.rename(columns={0:"food_int"})
df.shape

(1333, 10)

In [10]:
#give location type a number
location_model = OrdinalEncoder()
test = pd.DataFrame(location_model.fit_transform(df[['location']]))

df = pd.concat([df,test],axis=1,join='inner')
df = df.rename(columns={0:"location_int"})
df.shape

(1333, 11)

In [11]:
#introduce happy vs sad column
df['happy_vs_sad'] = df['happy']/df['sad']
df['happy_vs_sad'] = np.where(df['happy_vs_sad'] == np.inf, df['happy'], df['happy_vs_sad'])


<h1> Initial analysis

In [12]:
#General shape of our data
df.loc[:,['bookmark', 'happy', 'sad', 'price_int', 'happy_vs_sad']].describe()

bookmark        happy          sad    price_int  happy_vs_sad
count   1333.000000  1333.000000  1333.000000  1333.000000   1333.000000
mean   11943.156039   207.948237    11.912228   199.981245     39.039319
std    11846.389598   195.232354    17.067934   200.675706     49.418236
min      147.000000     3.000000     0.000000    25.000000      1.520833
25%     3790.000000    72.000000     2.000000    75.000000     11.685714
50%     8567.000000   141.000000     6.000000   150.000000     22.470588
75%    15675.000000   272.000000    15.000000   300.000000     46.000000
max    86503.000000  1238.000000   192.000000  1000.000000    701.000000

In [13]:
#location analysis, ppt
df.groupby(by='location').describe()

Unnamed: 0                                                   \
                   count         mean        std     min      25%     50%   
location                                                                    
Causeway Bay       235.0   126.157447  71.528644     0.0    65.00   127.0   
Central            204.0   596.151961  61.577069   491.0   543.75   594.5   
Mong Kok           232.0   370.232759  69.420153   249.0   310.75   370.5   
Tsim Sha Tsui      207.0   813.019324  63.827815   705.0   757.50   813.0   
Tsuen Wan          220.0  1275.804545  65.240146  1163.0  1219.75  1275.5   
Yuen Long          235.0  1042.587234  69.096581   924.0   983.50  1042.0   

                               bookmark                ... location_int       \
                   75%     max    count          mean  ...          75%  max   
location                                               ...                     
Causeway Bay    187.50   248.0    235.0  13615.893617  ...          0.0  0.0   
Central         647.25   704.0    204.0  12816.014706  ...          1.0  1.0   
Mong Kok        430.25   490.0    232.0  15327.327586  ...          2.0  2.0   
Tsim Sha Tsui   867.50   923.0    207.0  20531.516908  ...          3.0  3.0   
Tsuen Wan      1332.25  1387.0    220.0   5962.645455  ...          4.0  4.0   
Yuen Long      1101.50  1162.0    235.0   4205.442553  ...          5.0  5.0   

              happy_vs_sad                                             \
                     count       mean        std       min        25%   
location                                                                
Causeway Bay         235.0  47.613125  48.546323  3.750000  16.041667   
Central              204.0  39.732207  50.824018  2.611111  12.701389   
Mong Kok             232.0  44.323890  67.039495  3.212766  11.685119   
Tsim Sha Tsui        207.0  43.599394  53.994963  2.926829  13.000000   
Tsuen Wan            220.0  30.740390  35.286642  1.823529   9.185714   
Yuen Long            235.0  28.399382  28.923044  1.520833  10.000000   

                                            
                     50%        75%    max  
location                                    
Causeway Bay   28.500000  59.400000  347.0  
Central        22.055556  48.000000  345.5  
Mong Kok       22.639731  50.339286  701.0  
Tsim Sha Tsui  23.538462  46.979167  424.0  
Tsuen Wan      18.750000  36.600000  282.0  
Yuen Long      20.555556  34.000000  220.0  

[6 rows x 64 columns]

In [14]:
#find restaurant count in our dataset
restaurant_count = pd.DataFrame(df.groupby(by='food_type').count()['name'])

In [15]:
#dataframe of the 200 retaurants with highest happy count
top_200 = df.sort_values(by='happy_vs_sad', ascending = False).iloc[0:200,:]
top_200 = top_200.groupby(by='food_type').count()['name']
top_200 = pd.DataFrame(top_200)


In [16]:
#merge famous count and count of restaurant together
analysis = pd.DataFrame({'Food type': df['food_type'].unique()})
analysis = analysis.merge(top_200, left_on = 'Food type', right_on = 'food_type')
analysis = analysis.merge(restaurant_count, left_on = 'Food type', right_on = 'food_type')
analysis = analysis.rename(columns ={'name_x':'famous_count','name_y':'count'})


In [17]:
#calculate percentage of famous restaurant over frequency of said type of restaurant in dataset
analysis['percentage'] = analysis['famous_count']/analysis['count']*100
analysis.sort_values(by='famous_count',ascending = False).reset_index(drop=True)

Food type  famous_count  count  percentage
0          Japanese            52    330   15.757576
1   Hong Kong Style            35    168   20.833333
2           Western            33    225   14.666667
3     International            18     85   21.176471
4            Taiwan            16     82   19.512195
5         Guangdong            14     92   15.217391
6            Korean             6     58   10.344828
7              Thai             5     58    8.620690
8           Sichuan             4     36   11.111111
9        Vietnamese             3     19   15.789474
10          Italian             3     46    6.521739
11           Indian             2      8   25.000000
12          British             2      8   25.000000
13        Chiu Chow             1      9   11.111111
14           Yunnan             1      6   16.666667
15       Indonesian             1      4   25.000000
16          Spanish             1     12    8.333333
17      Singaporean             1      6   16.666667
18         American             1     17    5.882353
19         Peruvian             1      2   50.000000